**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML,Video
import skvideo
#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the next action the agent must take and have the current as argument. In training, the next action is either chosen randomly with probability epsilon or according to the running policy based on the state given in argument with probability 1-epsilon. The value of epsilon controls the tradoff between exploration and exploitation. It is essential to have an epsilon to perform random exploration to avoid getting stuck in local minimums.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1   #### Mistake original code self.position[-2:, :] = -1
 
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` is a grid_size x grid_size array used to represent the rewards that are present on the island. In ```reset``` the board is created by sampling malus and bonus ( two grid_size**2 random vectors) from two binomial distribution, multiplying by -1 and 0.5 for malus and bonus respectively reshaping and merging them. In case of conflict when merging the two vectors to create ```board```, a bonus is always chosen. Afterward, ```board``` is used in Environment ```act``` to read the reward based on position self.x | self.y of the agent. Since the rewards are collectible in the game, after reading the reward on a cell (x,y) its value is replaced by 0 ( also the case for the random initial position (x,y) in ```reset```). 

The array ```position``` is a grid_size x grid_size array used to represent the cell that the rat currently stands on (1) or not (0) and the boders of the island. In ```reset``` the ```position``` is initialized with a matrix of zeros and the margin are set to -1 ( the margin are 2 columns or 2 lines wide on all side of the square due to the 2 cells visibility, this is why we have grid size+=4 before initialization). In Environment ```act``` after reseting ```position``` to zeros, we set its cell self.x and self.y to 1 (same goes for the initial position in  ```reset```).

The state returned in ```reset``` and ```act``` is composed of the 5x5 submatrix of both ```board``` and ```position``` whose centers are given by the position of rat self.x | self.y.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:

class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # return a random integer between 0 and 3 (included) sampled from a uniform distribution 
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a 
    

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    
    score = 0
    won_games = 0
    
    for e in range(epochs):
        state = env.reset()  ## reset the state
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        won_games += int((win-lose)>0)
        score = score + win-lose

        print("Win/lose count {}/{}. Won games {}. Average score ({})"
              .format(win, lose, won_games, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/18.0. Won games 0. Average score (-5.5)
Win/lose count 8.0/13.0. Won games 0. Average score (-5.25)
Win/lose count 12.5/15.0. Won games 0. Average score (-4.333333333333333)
Win/lose count 10.5/9.0. Won games 1. Average score (-2.875)
Win/lose count 11.0/10.0. Won games 2. Average score (-2.1)
Win/lose count 6.5/10.0. Won games 2. Average score (-2.3333333333333335)
Win/lose count 14.5/17.0. Won games 2. Average score (-2.357142857142857)
Win/lose count 8.0/15.0. Won games 2. Average score (-2.9375)
Win/lose count 9.0/14.0. Won games 2. Average score (-3.1666666666666665)
Win/lose count 12.0/7.0. Won games 3. Average score (-2.35)
Win/lose count 10.5/19.0. Won games 3. Average score (-2.909090909090909)
Win/lose count 9.5/10.0. Won games 3. Average score (-2.7083333333333335)
Win/lose count 9.0/19.0. Won games 3. Average score (-3.269230769230769)
Win/lose count 9.0/20.0. Won games 3. Average score (-3.8214285714285716)
Win/lose count 7.0/13.0. Won games 3. Average 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




From the definition of the Q function we get:

\begin{equation*}
\begin{aligned}
Q^{\pi}(s,a) &= E_{p^{\pi}}[\sum_{t\leq T}\gamma^tr(s_t,a_t)|s_0=s,a_0=a] \\
&= E_{p^{\pi}}[r(s_0,a_0)+\gamma\sum_{1\leq t\leq T}\gamma^{t-1}r(s_t,a_t)|s_0=s,a_0=a] \\
&= E_{\pi(a_0)p(s_1|a_0,s_0)}[r(s_0,a_0)+\gamma E_{\pi(a_1|s1)\prod_{2\leq t\leq T}\pi(a_t|s_t)p(s_{t+1})|s_t,a_t }[\sum_{1\leq t\leq T}\gamma^{t-1}r(s_t,a_t)]|s_0=s,a_0=a]
\end{aligned}
\end{equation*}

Then,
\begin{equation*}
\begin{aligned}
E_{\pi(a_1|s1)\prod_{2\leq t\leq T}\pi(a_t|s_t)p(s_{t+1}|s_t,a_t) }[\sum_{1\leq t\leq T}\gamma^{t-1}r(s_t,a_t)] \\
&= E_{\pi(a_1|s_1)}[E_{\prod_{2\leq t\leq T}\pi(a_t|s_t)p(s_{t+1}|s_t,a_t }[\sum_{1\leq t\leq T}\gamma^{t-1}r(s_t,a_t) | a_1=a',s_1=s']]\\
&= E_{a'\sim \pi(.|s_1)}[Q^{\pi}(s_1,a')|s_1=s']
\end{aligned}
\end{equation*}


Thus,
\begin{equation*}
\begin{aligned}
Q^{\pi}(s,a)&= E_{\pi(a_0)p(s_1|a_0,s_0)}[r(s_0,a_0)+\gamma E_{a'\sim \pi(.|s_1)}[Q^{\pi}(s_1,a')|s_1=s']|a_0=a,s_0=s] \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
\end{equation*}

We can rewrite the condiational joint $p(s',a'|s,a) = p(s'|a,s)*\pi(a'|s') $ 

By definition :

\begin{equation*}
\begin{aligned}
Q^{*}(s,a) &= max_\pi E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\\ &= max_\pi E_{s'\sim p(.|s,a)}[r(s,a)+\gamma E_{a'\sim \pi(.|s')}[Q^{\pi}(s',a')]]
\\ &= E_{s'\sim p(.|s,a)}[r(s,a)+\gamma max_\pi E_{a'\sim \pi(.|s')}[Q^{\pi}(s',a')]]
\\ &= E_{s'\sim p(.|s,a)}[r(s,a)+\gamma max_\pi \sum_{a'} \pi(a'|s')Q^{\pi}(s',a')]
\\ &= E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{aligned}
\end{equation*}

In the deep RL setup we approximate the Q function for a given policy with a DNN. 
At temporal t and state $s_t$ if we perform $a_t$ according to the current policy we obtain the next state $s_t+1$ and the reward r. At training step i the goal is to train the DNN to approximate $Q(s_t,\theta_i)$, whose target for action are given by the recurrence relation derived previously. The targets for the other actions are set to 0. 

Which translates into:
\begin{equation*}
Q^*(s_t,a_t,\theta_i)= E_{s_{t+1}\sim p(.|s_t,a_t)}[r(s_t,a_t)+\gamma\max_{a'}Q^{*}(s_{t+1},a',\theta_{i-1})]
\end{equation*}
With our DNN having 4x1 outputs we use a norm to minimize the difference between those two expressions. Hence if for example $a_t = 3$: 

\begin{equation*}
L(\theta_i) = E_{s'\sim p(.|s,a)}\Vert \begin{bmatrix}
           Q(s_t, 1,\theta_i)-0 \\
           Q(s_t, 2,\theta_i)-0 \\
           Q(s_t, a_t,\theta_i)- r(s_t,a_t)+\gamma\max_{a'}Q(s_{t+1},a',\theta_{i-1})\\
           Q(s_t, 2,\theta_i)-0 \\
         \end{bmatrix} \Vert^2
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:       
            self.memory.append(m)  # Add the state to buffer 
        else:
            self.memory.pop(0) # popping the oldest state makes more sense than popping more recent states
            self.memory.append(m)

    def random_access(self):
        ## return a random state stored in the memory
        r_ind = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[r_ind]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    losses = []
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        losses.append(loss)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    return(score,loss,losses)

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [23]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        a = np.argmax(self.model.predict(s.reshape(1,5,5,self.n_state))) # a* = argmax_a Q(s,a)
        return a 

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # Create the batch from recovering batch_size steps stored in memory
        for i in range(self.batch_size):
            
            replay = self.memory.random_access()
            
            game_over_ = replay[-1]
            prev_state_,state_ = replay[0],replay[1]
            action_,reward_ = replay[2],replay[3]
            input_states[i,:,:] = replay[0]   # input state s_t 
            
            # Working with 4x1 vectors as targets/output of the DNN, one row per action
            # Thus the only non null element of target_q[0] is on the action_i-th row  
             
            if game_over_:
                target_q[i,action_] = reward_ 
            else:
                pred_state = self.model.predict(state_.reshape(1,5,5,self.n_state))
                target_q[i,action_] = reward_ + np.max(pred_state)
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -1, 1)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

    
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dense(units=256, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=4,activation='tanh'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

Given the relatively low complexity of the input, we stick to a standard architecture with 4 layers and relu activation functions except on the output layer where we use tanh (seems to be more effective). We use some dropout to prevent overfitting altough this is hardly an issue here. We use bayesian optimization to find out parameters that maximize our cumulated reward over all the epochs of training. We therefore use a memory size of 1365 and a lr of 0.16. The memory size must be high enough for the reinforcement step to be effective.

In [12]:
## Cell ran to find the params of the FC DNN, not to be ran again

#from skopt.space import Real, Integer
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#epochs_train = 20
#space  = [Integer(100, 2000, name='memory_size'),
#          Real(10**-5, 10**0, "log-uniform", name='lr'),
#          Integer(10, 300, name='batch_size')]
#
#env = Environment(grid_size=size, max_time=T, temperature=0.3)
#@use_named_args(space)
#def objective(**params):
#    agent = DQN_CNN(size,epsilon = 0.1,**params)
    #agent = DQN_FC(size,epsilon = 0.1,**params)

#    score,loss = train(agent, env, epochs_train, prefix='fc_train')
#    return -score

#res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)

#"Best score=%.4f" % res_gp.fun

In [25]:
epochs_train = 100
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.16, epsilon = 0.1, memory_size=1325, batch_size = 32)
score,loss,losses = train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train40.mp4'))

Epoch 000/100 | Loss 0.0079 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/100 | Loss 0.0072 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/100 | Loss 0.0042 | Win/lose count 3.5/6.0 (-2.5)
Epoch 003/100 | Loss 0.0062 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/100 | Loss 0.0027 | Win/lose count 2.5/8.0 (-5.5)
Epoch 005/100 | Loss 0.0059 | Win/lose count 2.5/6.0 (-3.5)
Epoch 006/100 | Loss 0.0080 | Win/lose count 4.0/8.0 (-4.0)
Epoch 007/100 | Loss 0.0157 | Win/lose count 5.0/6.0 (-1.0)
Epoch 008/100 | Loss 0.0025 | Win/lose count 1.0/4.0 (-3.0)
Epoch 009/100 | Loss 0.0007 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/100 | Loss 0.0184 | Win/lose count 4.5/6.0 (-1.5)
Epoch 011/100 | Loss 0.0065 | Win/lose count 5.0/3.0 (2.0)
Epoch 012/100 | Loss 0.0189 | Win/lose count 6.0/2.0 (4.0)
Epoch 013/100 | Loss 0.0029 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/100 | Loss 0.0109 | Win/lose count 5.0/5.0 (0.0)
Epoch 015/100 | Loss 0.0005 | Win/lose count 4.0/6.0 (-2.0)
Epoch 016/100 | Loss 0.0076 | Win/lose count 5

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [27]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(2, 2),input_shape=(5,5,self.n_state)))
        model.add(Conv2D(64, (2, 2)))
        model.add(Flatten())
        model.add(Dense(1000, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4,activation='tanh'))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

The architecture chosen is two 2D convolutional layers followed by one fully connected layer and some dropout. AvgPool, MaxPool and stride in conv2D are not considered here since the size of the grid 5x5 is too small. 

In [28]:
epochs_train = 100
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.16, epsilon = 0.1, memory_size=1325, batch_size = 50)
score,loss,losses_cnn = train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train40.mp4'))

Epoch 000/100 | Loss 0.0051 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/100 | Loss 0.0255 | Win/lose count 1.5/5.0 (-3.5)
Epoch 002/100 | Loss 0.0092 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/100 | Loss 0.0029 | Win/lose count 4.5/3.0 (1.5)
Epoch 004/100 | Loss 0.0065 | Win/lose count 2.5/2.0 (0.5)
Epoch 005/100 | Loss 0.0013 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/100 | Loss 0.0102 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/100 | Loss 0.0076 | Win/lose count 6.5/2.0 (4.5)
Epoch 008/100 | Loss 0.0025 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/100 | Loss 0.0038 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/100 | Loss 0.0053 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/100 | Loss 0.0086 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/100 | Loss 0.0031 | Win/lose count 5.5/1.0 (4.5)
Epoch 013/100 | Loss 0.0067 | Win/lose count 4.0/1.0 (3.0)
Epoch 014/100 | Loss 0.0019 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/100 | Loss 0.0012 | Win/lose count 2.0/3.0 (-1.0)
Epoch 016/100 | Loss 0.0037 | Win/lose count 2.0/2.0

Independantly from the hyperparameters or the scoring of the epoch, the final loss per epoch does not clearly converge. 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [29]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.16, epsilon = 0.1, memory_size=1325, batch_size = 262)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.16, epsilon = 0.1, memory_size=1325, batch_size = 262)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.5/0. Won games 1. Average score (6.5)
Win/lose count 6.0/2.0. Won games 2. Average score (5.25)
Win/lose count 6.0/1.0. Won games 3. Average score (5.166666666666667)
Win/lose count 7.0/1.0. Won games 4. Average score (5.375)
Win/lose count 8.5/1.0. Won games 5. Average score (5.8)
Win/lose count 12.0/4.0. Won games 6. Average score (6.166666666666667)
Win/lose count 13.0/2.0. Won games 7. Average score (6.857142857142857)
Win/lose count 4.5/2.0. Won games 8. Average score (6.3125)
Win/lose count 5.0/1.0. Won games 9. Average score (6.055555555555555)
Win/lose count 9.5/1.0. Won games 10. Average score (6.3)
Win/lose count 9.5/5.0. Won games 11. Average score (6.136363636363637)
Win/lose count 9.0/0. Won games 12. Average score (6.375)
Win/lose count 3.0/1.0. Won games 13. Average score (6.038461538461538)
Win/lose count 14.0/2.0. Won games 14. Average score (6.464285714285714)
Win/lose count 14.5/3.0. Won games 15. Average score (6.8)
Win/lose count 14

The states feeded to our networks are images with 'physical' meaning. Therefore, as expected CNN is much more effective than FC because conv2D is able to capture the topology of our input data, its symetries and invariances.

The temperature parameter controls to what extents the grid is densely populated with bonus and maluses. Knowing that during the creation of the board a bonus has priority over a malus, if the temperature is too high the board will have too much bonuses compared to maluses and the learning won't be relevant. On the contrary a low temperature will often result with our agent moving without purpose on a 'void' subgrid. Therefore, 0.3 seems like a good compromise.

Overall, the main issue limiting the performance is that the agent tends to restrict its movement to a subgrid of the map. This happens when its immediate environment does not have bonuses, often on the borders or corners. The agent does not have any incentive to get out of this subgrid, walking on already explored path is not penalized. 

In [30]:
HTML(display_videos('cnn_test10.mp4'))

In [31]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [34]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1   #### Mistake original code self.position[-2:, :] = -1
 
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y] ## Penalizing already explored cell in training mode
        reward += self.board[self.x, self.y]
        
        #if self.malus_position[self.x, self.y]<0.3:
        #    self.malus_position[self.x, self.y] += 0.1 ## Cumulative malus
        #else :
        #    self.malus_position[self.x, self.y] = self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position[self.x,self.y] = 0.1 # 
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [20]:
def train_explore(agent,env,epoch,prefix=''):
    
    # Number of won games
    score = 0
    loss = 0
    dis = 0.99
        
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
  
            agent.set_epsilon(agent.epsilon*dis) # Decreasing epsilon each time step
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win,3), round(lose,3), round(win-lose,3)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        


In [35]:
# Training
epochs_train = 100
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.4, memory_size=1300, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0096 | Win/lose count 5.0/24.0 (-19.0)
Epoch 001/100 | Loss 0.0068 | Win/lose count 6.0/20.9 (-14.9)
Epoch 002/100 | Loss 0.0089 | Win/lose count 6.1/19.2 (-13.1)
Epoch 003/100 | Loss 0.0077 | Win/lose count 6.5/19.0 (-12.5)
Epoch 004/100 | Loss 0.0083 | Win/lose count 4.9/18.9 (-14.0)
Epoch 005/100 | Loss 0.0033 | Win/lose count 4.5/21.4 (-16.9)
Epoch 006/100 | Loss 0.0068 | Win/lose count 3.2/20.3 (-17.1)
Epoch 007/100 | Loss 0.0133 | Win/lose count 7.9/23.9 (-16.0)
Epoch 008/100 | Loss 0.0069 | Win/lose count 7.3/19.5 (-12.2)
Epoch 009/100 | Loss 0.0043 | Win/lose count 6.0/23.3 (-17.3)
Epoch 010/100 | Loss 0.0034 | Win/lose count 6.1/19.6 (-13.5)
Epoch 011/100 | Loss 0.0051 | Win/lose count 5.6/23.6 (-18.0)
Epoch 012/100 | Loss 0.0071 | Win/lose count 3.2/19.3 (-16.1)
Epoch 013/100 | Loss 0.0115 | Win/lose count 3.6/20.2 (-16.6)
Epoch 014/100 | Loss 0.0094 | Win/lose count 3.8/20.0 (-16.2)
Epoch 015/100 | Loss 0.0108 | Win/lose count 6.4/20.3 (-13.9)
Epoch 01

In [36]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.5/2.0. Won games 1. Average score (12.5)
Win/lose count 11.0/1.0. Won games 2. Average score (11.25)
Win/lose count 17.0/3.0. Won games 3. Average score (12.166666666666666)
Win/lose count 16.5/0. Won games 4. Average score (13.25)
Win/lose count 15.5/1.0. Won games 5. Average score (13.5)
Win/lose count 7.0/0. Won games 6. Average score (12.416666666666666)
Win/lose count 18.5/3.0. Won games 7. Average score (12.857142857142858)
Win/lose count 11.5/0. Won games 8. Average score (12.6875)
Win/lose count 11.5/0. Won games 9. Average score (12.555555555555555)
Win/lose count 6.0/1.0. Won games 10. Average score (11.8)
Win/lose count 17.5/2.0. Won games 11. Average score (12.136363636363637)
Win/lose count 4.5/0. Won games 12. Average score (11.5)
Win/lose count 12.5/1.0. Won games 13. Average score (11.5)
Win/lose count 18.0/0. Won games 14. Average score (11.964285714285714)
Win/lose count 11.5/0. Won games 15. Average score (11.933333333333334)
Win/lose count 7.5/2.0.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***